In [1]:
%matplotlib inline
%load_ext rpy2.ipython

In [2]:
from skbio import TreeNode
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from math import log
from numpy import array
import sys
import re

In [3]:
%%R
library(ggplot2)
library(phyloseq)
library(gridExtra)
library(data.table)
library(vegan)
library(plyr);library(dplyr)
library(tidyr)
library(ape)

/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: data.table 1.9.6  For help type ?data.table or https://github.com/Rdatatable/data.table/wiki

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The fastest way to learn (by data.table authors): https://www.datacamp.com/courses/data-analysis-the-data-table-way

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: permute

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: lattice

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: This is vegan 2.3-5

  res =

In [4]:
#ghosttree = TreeNode.read("/var/seq_data/UNITE/trees/ghost_tree.nwk")  
ghosttree = TreeNode.read("data/ghost_tree_01_31_16_s_dynamic100.nwk")
output = open("ghost_tree_tips.txt", "w")

for node in ghosttree.tips():
    output.write(str(node.name)+"\n")

output.close()

In [ ]:
! head ghost_tree_tips.txt

In [ ]:
! grep -c "JX398982" ghost_tree_tips.txt

In [ ]:
! python python_script/parse_nonstandard_chars.py /var/seq_data/UNITE/31_01_16_s/sh_refs_qiime_ver7_dynamic_s_31.01.2016.fasta> /var/seq_data/UNITE/31_01_16_s/qiime_dynamic_nononstandchar.fasta

In [ ]:
! head /var/seq_data/UNITE/31_01_16_s/qiime_dynamic_nononstandchar.fasta

In [ ]:
! grep -c '>' /var/seq_data/UNITE/31_01_16_s/qiime_dynamic_nononstandchar.fasta

In [ ]:
! cat ghost_tree_tips.txt | sed -e 's/.*FU\(.*\)r.*/\1/' > acc.txt

In [ ]:
! head acc.txt

In [ ]:
! wc -l acc.txt

In [ ]:
! grep -c '>' /var/seq_data/UNITE/31_01_16_s/qiime_dynamic_nononstandchar.fasta

In [ ]:
f2 = open('acc.txt','r')
f1 = open('/var/seq_data/UNITE/31_01_16_s/qiime_dynamic_nononstandchar.fasta','r')
f3 = open('UNITE_fasta_parsed.fasta','w')



AI_DICT = {}
#for line in f2:
 #    f4 = line.strip()
        
for line in f2:
    line = line.strip()
    AI_DICT[line] = 1

skip = 0
for line in f1:
    if line.startswith('>'):
        #print line
        _splitline = line.split('_')
        accessorIDWithArrow = _splitline[1]
        accessorID = accessorIDWithArrow
        #print accessorID
        #break
        if accessorID in AI_DICT:
            f3.write(line)
            skip = 0
        else:
            skip = 1
    else:
        if not skip:
            pass
            f3.write(line)

f1.close()
f2.close()
f3.close()

In [ ]:
! grep -c '>' UNITE_fasta_parsed.fasta

In [ ]:
! grep -c 'JX398982' acc.txt

In [ ]:
! grep -c 'JX398982' UNITE_fasta_parsed.fasta

In [ ]:
! head UNITE_fasta_parsed.fasta

In [ ]:
from Bio.Blast.Applications import NcbiblastnCommandline as blast

In [ ]:
from cogent import LoadSeqs, DNA

In [ ]:
! grep -c '>' /home/chantal/Chazy/data/ComboFastaFiles/ITS/otusn.pick.fasta

In [ ]:
! makeblastdb -in UNITE_fasta_parsed.fasta -dbtype nucl -out UNITE_fasta_parsed.db

In [ ]:
cline = blast(query = "/home/chantal/Chazy/data/ComboFastaFiles/ITS/otusn.pick.fasta",
              db = "/home/chantal/Chazy/Bulk/ITS/UNITE_fasta_parsed.db",
              outfmt = 6,
              out = "/home/chantal/Chazy/Bulk/ITS/otusn_UNITE.m6",
              num_threads = 5)
print cline
cline()

In [ ]:
! head /home/chantal/Chazy/Bulk/ITS/otusn_UNITE.m6

In [5]:
!head -n1 otusn_UNITE.m6
!wc -l otusn_UNITE.m6
! grep 'OTU.10054' otusn_UNITE.m6

OTU.3	SH479820.07FU_KJ160145_reps_singleton	100.00	182	0	0	1	182	8	189	6e-93	  337
1197553 otusn_UNITE.m6


In [6]:
from IPython.display import Image
import pandas.rpy.common as com

/opt/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: The pandas.rpy module is deprecated and will be removed in a future version. We refer to external packages like rpy2. 
See here for a guide on how to port your code to rpy2: http://pandas.pydata.org/pandas-docs/stable/r_interface.html
  from ipykernel import kernelapp as app


In [7]:
%%R
library(data.table)
library(dplyr)

In [8]:
%%R
blastout_dt = fread("otusn_UNITE.m6")
blastout_df = tbl_df(blastout_dt)
colnames(blastout_df) = c("OTU",
                        "acc",
                        "pid",
                        "aln.length",
                        "mismatches",
                        "gapOpenCount",
                        "qstart",
                        "qend",
                        "sstart",
                        "send",
                        "e.value",
                        "bit.score")


Read 1197553 rows and 12 (of 12) columns from 0.091 GB file in 00:00:03


In [9]:
%%R
head(blastout_df) %>% as.data.frame

    OTU                                   acc    pid aln.length mismatches
1 OTU.3 SH479820.07FU_KJ160145_reps_singleton 100.00        182          0
2 OTU.3           SH443358.07FU_AY213653_refs 100.00        179          0
3 OTU.3 SH026903.07FU_AY354387_reps_singleton 100.00        179          0
4 OTU.3 SH458505.07FU_AY264267_reps_singleton  99.45        182          1
5 OTU.3 SH026901.07FU_JQ723341_reps_singleton  98.91        184          0
6 OTU.3 SH019632.07FU_JN400709_reps_singleton  98.91        184          0
  gapOpenCount qstart qend sstart send e.value bit.score
1            0      1  182      8  189   6e-93       337
2            0      4  182      1  179   3e-91       331
3            0      4  182      1  179   3e-91       331
4            0      1  182      9  190   3e-91       331
5            2      1  182      1  184   3e-90       327
6            2      1  182     18  201   3e-90       327


In [10]:
%%R
BS = blastout_df %>% group_by(OTU) %>% slice(which.min((bit.score)))

In [11]:
%%R
print(length(BS$OTU))
print(length(unique(BS$OTU)))

[1] 2686
[1] 2686


In [12]:
%%R
head(BS) %>% as.data.frame

        OTU                                   acc    pid aln.length mismatches
1   OTU.100 SH110535.07FU_AB481260_refs_singleton  82.11         95         11
2  OTU.1001           SH237405.07FU_JQ772020_refs  89.29         56          3
3 OTU.10077           SH185199.07FU_HQ630357_reps  92.31         39          0
4 OTU.10092           SH374767.07FU_AY236935_refs  89.66         58          3
5 OTU.10093 SH008011.07FU_FJ211865_reps_singleton  94.44         36          1
6 OTU.10096 SH134759.07FU_HQ179670_refs_singleton 100.00         37          0
  gapOpenCount qstart qend sstart send e.value bit.score
1            6     70  162    105  195   1e-14      76.8
2            3    174  226    165  220   1e-11      67.6
3            1     22   57      1   39   2e-07      52.8
4            3    120  175    167  223   6e-13      71.3
5            1    121  155    178  213   5e-08      54.7
6            0    232  268    225  261   4e-12      69.4


In [13]:
%%R 
summary(BS)

     OTU                acc                 pid           aln.length    
 Length:2686        Length:2686        Min.   : 73.60   Min.   : 28.00  
 Class :character   Class :character   1st Qu.: 90.91   1st Qu.: 34.00  
 Mode  :character   Mode  :character   Median : 96.88   Median : 41.00  
                                       Mean   : 94.43   Mean   : 50.19  
                                       3rd Qu.:100.00   3rd Qu.: 55.75  
                                       Max.   :100.00   Max.   :292.00  
   mismatches      gapOpenCount        qstart           qend      
 Min.   : 0.000   Min.   : 0.000   Min.   :  1.0   Min.   : 28.0  
 1st Qu.: 0.000   1st Qu.: 0.000   1st Qu.:122.0   1st Qu.:174.0  
 Median : 1.000   Median : 0.000   Median :156.0   Median :199.0  
 Mean   : 2.846   Mean   : 1.222   Mean   :145.2   Mean   :193.7  
 3rd Qu.: 3.000   3rd Qu.: 2.000   3rd Qu.:184.0   3rd Qu.:229.0  
 Max.   :58.000   Max.   :21.000   Max.   :313.0   Max.   :347.0  
     sstart         

In [14]:
%%R
OTUs_Acc = BS %>% ungroup() %>% select(OTU, acc)
head(OTUs_Acc) %>% as.data.frame
  


        OTU                                   acc
1   OTU.100 SH110535.07FU_AB481260_refs_singleton
2  OTU.1001           SH237405.07FU_JQ772020_refs
3 OTU.10077           SH185199.07FU_HQ630357_reps
4 OTU.10092           SH374767.07FU_AY236935_refs
5 OTU.10093 SH008011.07FU_FJ211865_reps_singleton
6 OTU.10096 SH134759.07FU_HQ179670_refs_singleton


In [15]:
%%R
print(length(OTUs_Acc$OTU))
print(length(unique(OTUs_Acc$acc)))

[1] 2686
[1] 729


In [16]:
! grep -c '>' /home/chantal/Chazy/data/ComboFastaFiles/ITS/otusn.pick.fasta

2882


In [17]:
%%R
newick<-function(text){
	text<-unlist(strsplit(text, NULL))
	tip.label<-vector(mode="character")
	node.label<-vector(mode="character") 
	edge<-matrix(c(1,NA),1,2) 
	edge.length<-vector()
	currnode<-1
	Nnode<-currnode
	i<-j<-k<-1
	while(text[i]!=";"){
		if(text[i]=="("){
			if(j>nrow(edge)) edge<-rbind(edge,c(NA,NA))
			edge[j,1]<-currnode
			i<-i+1
			# is the next element a label?
			if(is.na(match(text[i],c("(",")",",",":",";")))){
				temp<-getLabel(text,i)
				tip.label[k]<-temp$label
				i<-temp$end
				edge[j,2]<--k
				k<-k+1
				# is there a branch length?
				if(text[i]==":"){
					temp<-getEdgeLength(text,i)
					edge.length[j]<-temp$edge.length
					i<-temp$end
				}	
			} else if(text[i]=="("){
				Nnode<-Nnode+1 # creating a new internal node
				currnode<-Nnode
				edge[j,2]<-currnode # move to new internal node
			}
			j<-j+1
		} else if(text[i]==")"){
			i<-i+1
			# is the next element a label?
			if(is.na(match(text[i],c("(",")",",",":",";")))){
				temp<-getLabel(text,i)
				node.label[currnode]<-temp$label
				i<-temp$end
			}
			# is there a branch length?
			if(text[i]==":"){
				temp<-getEdgeLength(text,i)
				if(currnode>1){ 
					ii<-match(currnode,edge[,2])
					edge.length[ii]<-temp$edge.length
				} else root.edge<-temp$edge.length
				i<-temp$end
			}	
			if(currnode>1) currnode<-edge[match(currnode,edge[,2]),1] # move down the tree
		} else if(text[i]==","){
			if(j>nrow(edge)) edge<-rbind(edge,c(NA,NA))
			edge[j,1]<-currnode
			i<-i+1
			# is the next element a label?
			if(is.na(match(text[i],c("(",")",",",":",";")))){
				temp<-getLabel(text,i)
				tip.label[k]<-temp$label
				i<-temp$end
				edge[j,2]<--k
				k<-k+1
				# is there a branch length?
				if(text[i]==":"){
					temp<-getEdgeLength(text,i)
					edge.length[j]<-temp$edge.length
					i<-temp$end
				}
			} else if(text[i]=="("){
				Nnode<-Nnode+1 # creating a new internal node
				currnode<-Nnode
				edge[j,2]<-currnode # move to internal node
			}
			j<-j+1
		}
	}
	Ntip<-k-1
	edge[edge>0]<-edge[edge>0]+Ntip
	edge[edge<0]<--edge[edge<0]
	edge.length[is.na(edge.length)]<-0
	node.label[is.na(node.label)]<-""
	if(length(node.label)==0) node.label<-NULL
	# assemble into "phylo" object
	tree<-list(edge=edge,Nnode=as.integer(Nnode),tip.label=tip.label,edge.length=edge.length,node.label=node.label)
	class(tree)<-"phylo"
	return(tree)
}

In [18]:
%%R
# function gets label
# written by Liam J. Revell 2011-2013
getLabel<-function(text,start,stop.char=c(",",":",")")){
	i<-0
	label<-vector()
	while(is.na(match(text[i+start],stop.char))){
		label[i+1]<-text[i+start]
		i<-i+1
	}
	return(list(label=paste(label,collapse=""),end=i+start))
}

# function gets branch length
# written by Liam J. Revell 2011-2013
getEdgeLength<-function(text,start){
	i<-start+1; m<-1
	temp<-vector()
	while(is.na(match(text[i],c(",",")",";")))){
		temp[m]<-text[i]
		i<-i+1
		m<-m+1
	}
	return(list(edge.length=as.numeric(paste(temp,collapse="")),end=i))
}

In [19]:
%%R
read.newick<-function(file="",text){
# check to see if reading from file
    if(file!="") text<-scan(file,sep="\n",what="character")
    if(length(text)>1){
        tree<-lapply(text,newick)
        class(tree)<-"multiPhylo"
    } else tree<-newick(text)
    return(tree)
}
        
GT = read.newick('data/ghost_tree_01_31_16_s_dynamic100.nwk')

/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Read 1 item

  res = super(Function, self).__call__(*new_args, **new_kwargs)


In [20]:
%%R
Tips = as.data.frame(GT$tip.label)
colnames(Tips)= 'tips'
print(head(Tips))
print(length(Tips$tips))

                         tips
1 SH221552.07FU_AB738885_refs
2 SH214405.07FU_AB699657_refs
3 SH214407.07FU_AB738881_reps
4 SH214406.07FU_AB738883_refs
5 SH200583.07FU_AB699660_refs
6 SH202218.07FU_KC152249_reps
[1] 21710


In [21]:
%%R
print(head(OTUs_Acc))
print(length(unique(OTUs_Acc$acc)))
print(length(unique(OTUs_Acc$OTU)))

Source: local data frame [6 x 2]

        OTU                                   acc
      (chr)                                 (chr)
1   OTU.100 SH110535.07FU_AB481260_refs_singleton
2  OTU.1001           SH237405.07FU_JQ772020_refs
3 OTU.10077           SH185199.07FU_HQ630357_reps
4 OTU.10092           SH374767.07FU_AY236935_refs
5 OTU.10093 SH008011.07FU_FJ211865_reps_singleton
6 OTU.10096 SH134759.07FU_HQ179670_refs_singleton
[1] 729
[1] 2686


In [22]:
%%R
print(length(Tips$tips))

[1] 21710


In [23]:
%%R
test = as.data.frame(GT$tip.label)
head(test)

                 GT$tip.label
1 SH221552.07FU_AB738885_refs
2 SH214405.07FU_AB699657_refs
3 SH214407.07FU_AB738881_reps
4 SH214406.07FU_AB738883_refs
5 SH200583.07FU_AB699660_refs
6 SH202218.07FU_KC152249_reps


In [24]:
%%R
testmatch = match(OTUs_Acc$acc, GT$tip.label, nomatch = NA_integer_, incomparables = NULL)


In [25]:
%%R
head(testmatch)

[1] 14720  8478 18041  8360  5161 21415


In [26]:
%%R
OTUs_Acc$TreeTipPos = testmatch
head(OTUs_Acc)

Source: local data frame [6 x 3]

        OTU                                   acc TreeTipPos
      (chr)                                 (chr)      (int)
1   OTU.100 SH110535.07FU_AB481260_refs_singleton      14720
2  OTU.1001           SH237405.07FU_JQ772020_refs       8478
3 OTU.10077           SH185199.07FU_HQ630357_reps      18041
4 OTU.10092           SH374767.07FU_AY236935_refs       8360
5 OTU.10093 SH008011.07FU_FJ211865_reps_singleton       5161
6 OTU.10096 SH134759.07FU_HQ179670_refs_singleton      21415


In [27]:
%%R
OTUs_filt = filter(OTUs_Acc, !(is.na(TreeTipPos)))
length(OTUs_filt$OTU)

[1] 2686


In [28]:
%%R
test = GT$tip.label
test[8095]

[1] "SH216793.07FU_FJ357315_refs"


In [29]:
%%R 
for(line in OTUs_filt){
    test[line] = OTUs_filt[line, 3]
    }

In [30]:
%%R
length(unique(OTUs_filt$TreeTipPos))

[1] 729


In [31]:
%%R
OTUs_filt %>% group_by(TreeTipPos) %>% summarise(OTUnum = n()) %>% filter(OTUnum >1) %>% arrange(-OTUnum)

Source: local data frame [296 x 2]

   TreeTipPos OTUnum
        (int)  (int)
1        9440     67
2       15197     63
3       21684     59
4       19914     57
5       10182     54
6        9417     51
7       15611     46
8        6578     34
9       11186     34
10       4932     33
..        ...    ...


In [32]:
%%R
filter(OTUs_filt, TreeTipPos == 9440)

Source: local data frame [67 x 3]

         OTU                         acc TreeTipPos
       (chr)                       (chr)      (int)
1  OTU.11845 SH446992.07FU_KC254054_refs       9440
2   OTU.1199 SH446992.07FU_KC254054_refs       9440
3   OTU.1230 SH446992.07FU_KC254054_refs       9440
4  OTU.12356 SH446992.07FU_KC254054_refs       9440
5   OTU.1240 SH446992.07FU_KC254054_refs       9440
6  OTU.13430 SH446992.07FU_KC254054_refs       9440
7  OTU.15761 SH446992.07FU_KC254054_refs       9440
8  OTU.16216 SH446992.07FU_KC254054_refs       9440
9  OTU.16498 SH446992.07FU_KC254054_refs       9440
10   OTU.173 SH446992.07FU_KC254054_refs       9440
..       ...                         ...        ...


In [33]:
%%R
treetips = as.data.frame(GT$tip.label)
colnames(treetips) = 'tip.label'
treetips = filter(treetips, !(tip.label %in% OTUs_filt$acc))
#treetips = head(treetips, n = 15000)

In [34]:
%%R
length(treetips$tip.label)

[1] 20981


In [67]:
%%R 
GT2 = collapse.singles(GT)

In [68]:
%%R -h 1000 -w 700
GT3 = drop.tip(GT2, as.vector(treetips$tip.label))

In [69]:
%%R
write.tree(GT3, 'data/filt_ghosttree.nwk')

In [37]:
%%R
NT_Ret = read.csv('data/NT_Ret_responders.csv')
NT_Ret = select(NT_Ret, OTU, LFC_Till, p_Till, LFC_OM, p_OM, Rank1.x, Rank2.x, Rank3.x, Rank4.x, Rank5.x, Till, OM)

In [38]:
%%R
print(head(NT_Ret))
print(head(OTUs_filt))

       OTU  LFC_Till       p_Till   LFC_OM         p_OM  Rank1.x
1 OTU.4807 -2.243068 2.195440e-03 2.715466 5.891434e-04 k__Fungi
2 OTU.1259 -3.151002 2.338111e-06 2.290393 4.805899e-03 k__Fungi
3 OTU.1434 -2.365286 5.868299e-10 1.272158 1.258542e-02 k__Fungi
4  OTU.192 -1.814900 8.355789e-11 1.630828 2.174634e-07 k__Fungi
5 OTU.5253 -4.943473 9.764712e-10 4.729240 1.572899e-08 k__Fungi
6 OTU.2624 -1.598050 9.793366e-02 3.090019 7.436505e-04 k__Fungi
           Rank2.x            Rank3.x                Rank4.x            Rank5.x
1 p__Basidiomycota  c__Agaricomycetes          o__Agaricales f__Psathyrellaceae
2 p__Basidiomycota  c__Agaricomycetes          o__Agaricales f__Psathyrellaceae
3 p__Basidiomycota  c__Agaricomycetes          o__Agaricales f__Psathyrellaceae
4    p__Ascomycota c__Sordariomycetes         o__Hypocreales    f__Hypocreaceae
5    p__Ascomycota               <NA>                   <NA>               <NA>
6 p__Basidiomycota c__Tremellomycetes o__Cystofilobasidiales  f__

In [39]:
%%R
length(NT_Ret$OTU)

[1] 69


In [40]:
%%R
OTUs_NT_Ret = left_join(NT_Ret, OTUs_filt)


/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Joining by: "OTU"

  res = super(Function, self).__call__(*new_args, **new_kwargs)


### Using Nick's Expand function to add polytomies to tree for OTUs that map to same Ghosttree tip

* For expand function, need tree file (newick), count file in Mothur format, -f specifies tree file format

In [43]:
%%R
tips = as.data.frame(GT$tip.label)
colnames(tips) = 'tipname'
head(tips)

                      tipname
1 SH221552.07FU_AB738885_refs
2 SH214405.07FU_AB699657_refs
3 SH214407.07FU_AB738881_reps
4 SH214406.07FU_AB738883_refs
5 SH200583.07FU_AB699660_refs
6 SH202218.07FU_KC152249_reps


In [51]:
%%R
OTUs_acc = select(OTUs_filt, OTU, acc) 
OTUs_acc = plyr::rename(OTUs_acc, replace = c('acc' = 'tipname'))

head(OTUs_acc)

Source: local data frame [6 x 2]

        OTU                               tipname
      (chr)                                 (chr)
1   OTU.100 SH110535.07FU_AB481260_refs_singleton
2  OTU.1001           SH237405.07FU_JQ772020_refs
3 OTU.10077           SH185199.07FU_HQ630357_reps
4 OTU.10092           SH374767.07FU_AY236935_refs
5 OTU.10093 SH008011.07FU_FJ211865_reps_singleton
6 OTU.10096 SH134759.07FU_HQ179670_refs_singleton


In [85]:
%%R
filter(OTUs_acc, tipname == 'SH221552.07FU_AB738885_refs')

Source: local data frame [0 x 4]

Variables not shown: OTU (chr), tipname (chr), val (dbl), proxy (dbl)


In [81]:
%%R
OTUs_acc$val = 1
OTUs_acc$proxy = 100
OTUs_spread = spread(OTUs_acc, OTU, val, fill = 0)

write.table(OTUs_spread, sep = '\t', file = 'data/otuTable_ghosttree.txt', row.names = FALSE)

In [83]:
! /home/chantal/tree_edit/bin/expand.pl -t data/filt_ghosttree.nwk -c data/otuTable_ghosttree.txt > data/polytomy_Ghostree.nwk

 Newick tree file written:
  data/filt_ghosttree_exp.nwk


In [84]:
! /home/chantal/tree_edit/bin/expand.pl -t data/ghost_tree_01_31_16_s_dynamic100.nwk -c data/otuTable_ghosttree.txt > data/polytomy_Ghostree_full.nwk

 ERROR: SH221552.07FU_AB738885_refs not found in count table


### Once have tree with polytomy, could map traits..., use in phyloseq??